In [58]:
GYRO = False
SEQUENCE_LENGTH = 5
SEQUENCE_OVERLAP = 4
BATCH_SIZE = 10
EPOCHS = 100
MODEL_NAME = f"physical_activity_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}.keras"

TRAINING_SIZE = 0.8
DEV_SIZE = 0.1
TEST_SIZE = 0.1

# EDIT THE VALUES IN THE CELL ABOVE THEN PRESS RUN ALL

In [59]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential
from sklearn.model_selection import train_test_split


In [60]:
DATA_DIRECTORY = "./all_respeck"
PHYSICAL_ACTIVITIES = {
    "sitting&normal_breathing",
    "standing&normal_breathing",
    "lying_down_left&normal_breathing",
    "lying_down_right&normal_breathing",
    "lying_down_back&normal_breathing",
    "lying_down_stomach&normal_breathing",
    "walking&normal_breathing",
    "ascending_stairs&normal_breathing",
    "descending_stairs&normal_breathing",
    "shuffle_walking&normal_breathing",
    "running&normal_breathing",
    "misc_movements&normal_breathing"
}

In [61]:
def generate_training_data(directory, sequence_length, overlap, gyro = GYRO): # if gyro is false, only accelerometer data is used
    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in PHYSICAL_ACTIVITIES:

        # iterates through each csv file for the activity 
        for csv_file in csv_dictionary[key]:
            if gyro:
                sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
            else:
                sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

            # iterate through each generated sequence
            for sequence in sequences:
                tagged_data.append((key, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [62]:
# split data into training, dev, and test sets
def train_dev_test_split(data, labels, dev_size, test_size, random_state=42):
    # Split the data into training and temporary (dev + test) sets
    train_data, temp_data, train_labels, temp_labels = train_test_split(data, labels, test_size=(dev_size + test_size), random_state=random_state)
    
    # Split the temporary data into dev and test sets
    dev_data, test_data, dev_labels, test_labels = train_test_split(temp_data, temp_labels, 
                                                                 test_size=(test_size / (dev_size + test_size)), random_state=random_state)
    
    return train_data, dev_data, test_data, train_labels, dev_labels, test_labels

In [63]:
#CNN MODEL
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [64]:
tagged_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP)

# Get lists of training data and labels
sequences = [sequence for _, sequence in tagged_sequences]
labels = [label for label, _ in tagged_sequences]


# encode labels to numbers
sequences = np.array(sequences, dtype=np.float32)
label_to_index = {label: idx for idx, label in enumerate(PHYSICAL_ACTIVITIES)}
labels_encoded = [label_to_index[label] for label in labels]
labels_encoded = np.array(labels_encoded)

train_data, dev_data, test_data, train_labels, dev_labels, test_labels = train_dev_test_split(sequences, labels_encoded, dev_size=DEV_SIZE, test_size=TEST_SIZE) #10% dev, 10% test

# train and save model (CHOOSE BETWEEN CNN AND LSTM)
model = train_model_CNN(train_data, train_labels, PHYSICAL_ACTIVITIES, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(dev_data, dev_labels)) #batch_size, epochs


# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the trained model
model.save(f"models/Task1/{MODEL_NAME}_{test_accuracy}")

there are 14326 tagged sequences in the dataset
Epoch 1/100
1146/1146 [==============================] - 5s 4ms/step - loss: 0.9538 - accuracy: 0.6552 - val_loss: 0.5651 - val_accuracy: 0.7760
Epoch 2/100
1146/1146 [==============================] - 4s 3ms/step - loss: 0.5039 - accuracy: 0.8129 - val_loss: 0.3333 - val_accuracy: 0.8674
Epoch 3/100
1146/1146 [==============================] - 4s 4ms/step - loss: 0.3735 - accuracy: 0.8545 - val_loss: 0.2812 - val_accuracy: 0.8925
Epoch 4/100
1146/1146 [==============================] - 4s 4ms/step - loss: 0.3000 - accuracy: 0.8798 - val_loss: 0.2455 - val_accuracy: 0.8911
Epoch 5/100
1146/1146 [==============================] - 4s 4ms/step - loss: 0.2600 - accuracy: 0.8931 - val_loss: 0.2353 - val_accuracy: 0.9093
Epoch 6/100
1146/1146 [==============================] - 4s 4ms/step - loss: 0.2483 - accuracy: 0.8972 - val_loss: 0.2109 - val_accuracy: 0.9107
Epoch 7/100
1146/1146 [==============================] - 4s 4ms/step - loss: 0.224

2023-11-03 19:07:58.124994: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,5,128]
	 [[{{node inputs}}]]
2023-11-03 19:07:58.467272: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,5,128]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: models/Task1/physical_activity_epochs:100_batch:10_gyro:False_window:5_overlap:4.keras_0.951151430606842/assets


INFO:tensorflow:Assets written to: models/Task1/physical_activity_epochs:100_batch:10_gyro:False_window:5_overlap:4.keras_0.951151430606842/assets
